In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pyperclip
from deep_translator import GoogleTranslator
import pyttsx3
import logging
import time
import os
from langdetect import detect

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Initialize text-to-speech engine
tts_engine = pyttsx3.init()
tts_speaking = False
undo_stack = []
favorites = []
is_dark_mode = False

# Translation function
def translate_text():
    global tts_speaking
    input_text = input_text_area.get(1.0, tk.END).strip()
    target_lang = lang_map.get(lang_dropdown.get(), "en")
    source_lang = 'auto'

    if not input_text:
        messagebox.showwarning("No Text", "Please enter text to translate.")
        return
    if len(input_text) > 500000:
        messagebox.showwarning("Text Too Long", "Input characters.")
        return

    if auto_detect_var.get():
        try:
            source_lang = detect(input_text)
            logger.info(f"Detected source language: {source_lang}")
        except Exception as e:
            logger.warning(f"Language detection failed: {e}")
            source_lang = 'auto'

    logger.info(f"Translating '{input_text[:20]}...' from {source_lang} to {target_lang}")
    for attempt in range(3):
        try:
            translator = GoogleTranslator(source=source_lang, target=target_lang)
            translated = translator.translate(input_text)
            if not translated:
                raise ValueError("Empty translation result")
            # Save current state for undo
            undo_stack.append((input_text_area.get(1.0, tk.END).strip(), output_text_area.get(1.0, tk.END).strip(), lang_dropdown.get()))
            output_text_area.config(state=tk.NORMAL)
            output_text_area.delete(1.0, tk.END)
            output_text_area.insert(tk.END, translated)
            output_text_area.config(state=tk.DISABLED)
            copy_button.config(state=tk.NORMAL)
            speak_button.config(state=tk.NORMAL)
            undo_button.config(state=tk.NORMAL)
            history.append((input_text[:20], translated[:20]))
            update_history_button()
            # Estimate confidence (heuristic: length similarity)
            confidence = min(len(translated) / len(input_text), len(input_text) / len(translated)) if len(translated) > 0 else 0
            confidence = max(0.5, min(confidence, 1.0)) * 100
            confidence_label.config(text=f"Confidence: {confidence:.1f}%")
            logger.info("Translation successful")
            return
        except Exception as e:
            logger.warning(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(1)
    logger.error("All translation attempts failed")
    messagebox.showerror("Error", "Translation failed. Check your connection or try again.")
    output_text_area.config(state=tk.NORMAL)
    output_text_area.delete(1.0, tk.END)
    output_text_area.insert(tk.END, "Translation unavailable")
    output_text_area.config(state=tk.DISABLED)
    confidence_label.config(text="Confidence: N/A")
    tts_speaking = False

# Undo translation
def undo_translation():
    if undo_stack:
        prev_input, prev_output, prev_lang = undo_stack.pop()
        input_text_area.delete(1.0, tk.END)
        input_text_area.insert(tk.END, prev_input)
        output_text_area.config(state=tk.NORMAL)
        output_text_area.delete(1.0, tk.END)
        output_text_area.insert(tk.END, prev_output)
        output_text_area.config(state=tk.DISABLED if prev_output else tk.NORMAL)
        lang_dropdown.set(prev_lang)
        copy_button.config(state=tk.NORMAL if prev_output and prev_output != "Translation unavailable" else tk.DISABLED)
        speak_button.config(state=tk.NORMAL if prev_output and prev_output != "Translation unavailable" else tk.DISABLED)
        undo_button.config(state=tk.NORMAL if undo_stack else tk.DISABLED)
        confidence_label.config(text="Confidence: N/A")

# Clear input and output
def clear_text():
    global tts_speaking
    input_text_area.delete(1.0, tk.END)
    output_text_area.config(state=tk.NORMAL)
    output_text_area.delete(1.0, tk.END)
    output_text_area.config(state=tk.DISABLED)
    copy_button.config(state=tk.DISABLED)
    speak_button.config(state=tk.DISABLED)
    confidence_label.config(text="Confidence: N/A")
    tts_speaking = False
    tts_engine.stop()

# Copy translated text to clipboard
def copy_text():
    translated_text = output_text_area.get(1.0, tk.END).strip()
    if translated_text and translated_text != "Translation unavailable":
        pyperclip.copy(translated_text)
        messagebox.showinfo("Copied", "Text copied to clipboard!")

# Toggle text-to-speech
def toggle_speech():
    global tts_speaking
    translated_text = output_text_area.get(1.0, tk.END).strip()
    if translated_text and translated_text != "Translation unavailable":
        if not tts_speaking:
            tts_speaking = True
            speak_button.config(text="Stop Speaking")
            tts_engine.say(translated_text)
            tts_engine.runAndWait()
            tts_speaking = False
            speak_button.config(text="Speak")
        else:
            tts_speaking = False
            tts_engine.stop()
            speak_button.config(text="Speak")

# Export translation to text file
def export_translation():
    translated_text = output_text_area.get(1.0, tk.END).strip()
    if translated_text and translated_text != "Translation unavailable":
        file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if file_path:
            try:
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(translated_text)
                messagebox.showinfo("Success", "Translation exported successfully!")
            except Exception as e:
                logger.error(f"Export failed: {e}")
                messagebox.showerror("Error", "Failed to export translation.")

# Manage favorites
def add_to_favorites():
    current_lang = lang_dropdown.get()
    if current_lang not in favorites:
        favorites.append(current_lang)
        favorites_dropdown['values'] = favorites
        messagebox.showinfo("Favorites", f"Added {current_lang} to favorites!")
    else:
        messagebox.showinfo("Favorites", f"{current_lang} is already in favorites.")

def load_favorite(event):
    selected_favorite = favorites_dropdown.get()
    if selected_favorite:
        lang_dropdown.set(selected_favorite)

# Toggle theme
def toggle_theme():
    global is_dark_mode
    is_dark_mode = not is_dark_mode
    if is_dark_mode:
        colors.update({
            "primary": "#4dabf7",
            "primary_hover": "#339af0",
            "secondary": "#adb5bd",
            "secondary_hover": "#868e96",
            "accent": "#51cf66",
            "accent_hover": "#37b24d",
            "danger": "#ff8787",
            "danger_hover": "#ff6b6b",
            "speak": "#22b8cf",
            "speak_hover": "#15aabf",
            "input_bg": "#343a40",
            "output_bg": "#212529",
            "canvas_bg": "#1a1d21",
            "frame_bg": "#1a1d21"
        })
        header_label.config(fg="#e9ecef")
        input_label.config(fg="#e9ecef")
        output_label.config(fg="#e9ecef")
        lang_label.config(fg="#e9ecef")
        confidence_label.config(fg="#e9ecef")
        theme_button.config(text="Light Mode")
    else:
        colors.update({
            "primary": "#007bff",
            "primary_hover": "#0056b3",
            "secondary": "#6c757d",
            "secondary_hover": "#545b62",
            "accent": "#28a745",
            "accent_hover": "#218838",
            "danger": "#ff4d4d",
            "danger_hover": "#cc0000",
            "speak": "#17a2b8",
            "speak_hover": "#117a8b",
            "input_bg": "#e9ecef",
            "output_bg": "#ffffff",
            "canvas_bg": "#f0f2f5",
            "frame_bg": "#f0f2f5"
        })
        header_label.config(fg="#343a40")
        input_label.config(fg="#495057")
        output_label.config(fg="#495057")
        lang_label.config(fg="#495057")
        confidence_label.config(fg="#495057")
        theme_button.config(text="Dark Mode")
    root.config(bg=colors["canvas_bg"])
    scroll_canvas.config(bg=colors["canvas_bg"])
    content.config(bg=colors["frame_bg"])
    input_frame.config(bg=colors["frame_bg"])
    action_frame.config(bg=colors["frame_bg"])
    output_frame.config(bg=colors["frame_bg"])
    input_text_area.config(bg=colors["input_bg"], fg="#ffffff" if is_dark_mode else "#212529")
    output_text_area.config(bg=colors["output_bg"], fg="#ffffff" if is_dark_mode else "#212529")
    translate_button.config(bg=colors["primary"])
    copy_button.config(bg=colors["accent"])
    speak_button.config(bg=colors["speak"])
    clear_button.config(bg=colors["secondary"])
    history_button.config(bg=colors["danger"])
    export_button.config(bg=colors["primary"])
    undo_button.config(bg=colors["secondary"])
    theme_button.config(bg=colors["accent"])

# Show translation history in a popup
def show_history():
    popup = tk.Toplevel(root)
    popup.title("Translation History")
    popup.geometry("600x500")
    popup.configure(bg="#ffffff" if not is_dark_mode else "#212529")
    popup.transient(root)
    popup.grab_set()

    listbox = tk.Listbox(popup, font=("Helvetica", 14), bg="#f7f9fc" if not is_dark_mode else "#343a40", fg="#212529" if not is_dark_mode else "#e9ecef", bd=0, relief="flat")
    listbox.pack(fill="both", expand=True, padx=20, pady=20)
    for i, (inp, out) in enumerate(history):
        listbox.insert(tk.END, f"{i+1}. {inp} -> {out}")

    close_btn = tk.Button(popup, text="Close", command=popup.destroy, bg="#ff4d4d", fg="white", font=("Helvetica", 12), relief="flat", bd=0)
    close_btn.pack(pady=10)

# Update history button state
def update_history_button():
    history_button.config(state="normal" if history else "disabled")

# Button hover effects
def on_enter(btn, hover_color):
    btn.config(bg=hover_color)

def on_leave(btn, base_color):
    btn.config(bg=base_color)

# Update scroll region
def update_scroll_region(event=None):
    scroll_canvas.configure(scrollregion=scroll_canvas.bbox("all"))

# Mouse wheel scrolling
def on_mousewheel(event):
    scroll_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

# Initialize Tkinter window
root = tk.Tk()
root.title("Modern Translator")
root.state('zoomed')
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
root.geometry(f"{screen_width}x{screen_height}+0+0")
root.resizable(True, True)
root.configure(bg="#f0f2f5")

# Fonts and colors
font_large = ("Helvetica", 20, "bold")
font_medium = ("Helvetica", 16)
font_small = ("Helvetica", 14)
colors = {
    "primary": "#007bff",
    "primary_hover": "#0056b3",
    "secondary": "#6c757d",
    "secondary_hover": "#545b62",
    "accent": "#28a745",
    "accent_hover": "#218838",
    "danger": "#ff4d4d",
    "danger_hover": "#cc0000",
    "speak": "#17a2b8",
    "speak_hover": "#117a8b",
    "input_bg": "#e9ecef",
    "output_bg": "#ffffff",
    "canvas_bg": "#f0f2f5",
    "frame_bg": "#f0f2f5"
}

# Language mapping (20 languages)
lang_map = {
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Arabic": "ar",
    "Italian": "it",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh-cn",
    "Russian": "ru",
    "Portuguese": "pt",
    "Korean": "ko",
    "Dutch": "nl",
    "Swedish": "sv",
    "Hindi": "hi",
    "Turkish": "tr",
    "Polish": "pl",
    "Greek": "el",
    "Thai": "th",
    "Vietnamese": "vi",
    "Hebrew": "he"
}

# History storage
history = []

# Scrollable canvas for entire interface
scroll_canvas = tk.Canvas(root, bg=colors["canvas_bg"], highlightthickness=0)
scrollbar = tk.Scrollbar(root, orient="vertical", command=scroll_canvas.yview)
scroll_canvas.configure(yscrollcommand=scrollbar.set)
scrollbar.pack(side="right", fill="y")
scroll_canvas.pack(side="left", fill="both", expand=True)

# Content frame
content = tk.Frame(scroll_canvas, bg=colors["frame_bg"], padx=40, pady=40)
content_window = scroll_canvas.create_window(screen_width // 2, 0, window=content, anchor="n")
content.bind("<Configure>", update_scroll_region)
scroll_canvas.bind_all("<MouseWheel>", on_mousewheel)

# Header label
header_label = tk.Label(content, text="Translator", font=("Helvetica", 24, "bold"), bg=colors["frame_bg"], fg="#343a40")
header_label.grid(row=0, column=0, columnspan=2, pady=(0, 20))

# Toolbar for advanced options
toolbar_frame = tk.Frame(content, bg=colors["frame_bg"])
toolbar_frame.grid(row=1, column=0, columnspan=2, pady=10, sticky="ew")

auto_detect_var = tk.BooleanVar()
auto_detect_check = tk.Checkbutton(toolbar_frame, text="Auto-Detect Language", variable=auto_detect_var, font=font_small, bg=colors["frame_bg"], fg="#495057")
auto_detect_check.pack(side="left", padx=10)

favorites_label = tk.Label(toolbar_frame, text="Favorites:", font=font_small, bg=colors["frame_bg"], fg="#495057")
favorites_label.pack(side="left", padx=(20, 5))
favorites_dropdown = ttk.Combobox(toolbar_frame, values=favorites, font=font_small, state="readonly", width=15)
favorites_dropdown.pack(side="left")
favorites_dropdown.bind("<<ComboboxSelected>>", load_favorite)
add_favorite_button = tk.Button(toolbar_frame, text="Add Favorite", command=add_to_favorites, font=font_small, bg=colors["secondary"], fg="white", relief="flat", bd=0)
add_favorite_button.pack(side="left", padx=10)
add_favorite_button.bind("<Enter>", lambda e: on_enter(add_favorite_button, colors["secondary_hover"]))
add_favorite_button.bind("<Leave>", lambda e: on_leave(add_favorite_button, colors["secondary"]))

export_button = tk.Button(toolbar_frame, text="Export", command=export_translation, font=font_small, bg=colors["primary"], fg="white", relief="flat", bd=0)
export_button.pack(side="left", padx=10)
export_button.bind("<Enter>", lambda e: on_enter(export_button, colors["primary_hover"]))
export_button.bind("<Leave>", lambda e: on_leave(export_button, colors["primary"]))

undo_button = tk.Button(toolbar_frame, text="Undo", command=undo_translation, font=font_small, bg=colors["secondary"], fg="white", relief="flat", bd=0, state="disabled")
undo_button.pack(side="left", padx=10)
undo_button.bind("<Enter>", lambda e: on_enter(undo_button, colors["secondary_hover"]))
undo_button.bind("<Leave>", lambda e: on_leave(undo_button, colors["secondary"]))

theme_button = tk.Button(toolbar_frame, text="Dark Mode", command=toggle_theme, font=font_small, bg=colors["accent"], fg="white", relief="flat", bd=0)
theme_button.pack(side="left", padx=10)
theme_button.bind("<Enter>", lambda e: on_enter(theme_button, colors["accent_hover"]))
theme_button.bind("<Leave>", lambda e: on_leave(theme_button, colors["accent"]))

# Input section
input_frame = tk.Frame(content, bg=colors["frame_bg"])
input_frame.grid(row=2, column=0, columnspan=2, sticky="ew")
input_label = tk.Label(input_frame, text="Enter Text", font=font_medium, bg=colors["frame_bg"], fg="#495057")
input_label.pack(anchor="w")
input_text_area = tk.Text(input_frame, wrap=tk.WORD, height=8, width=80, font=font_small, bg=colors["input_bg"], fg="#212529", relief="flat", bd=1, highlightthickness=1, highlightbackground="#ced4da")
input_text_area.pack(fill="x", pady=10)
input_scroll = tk.Scrollbar(input_frame, orient="vertical", command=input_text_area.yview)
input_text_area.config(yscrollcommand=input_scroll.set)
input_scroll.pack(side="right", fill="y")

# Language and action frame
action_frame = tk.Frame(content, bg=colors["frame_bg"])
action_frame.grid(row=3, column=0, columnspan=2, pady=20)
lang_label = tk.Label(action_frame, text="Select Language", font=font_medium, bg=colors["frame_bg"], fg="#495057")
lang_label.grid(row=0, column=0, padx=(0, 20))
lang_dropdown = ttk.Combobox(action_frame, values=list(lang_map.keys()), font=font_small, state="readonly", width=20)
lang_dropdown.set("English")
lang_dropdown.grid(row=0, column=1)

# Buttons
translate_button = tk.Button(action_frame, text="Translate", command=translate_text, font=font_medium, bg=colors["primary"], fg="white", relief="flat", bd=0, width=12)
translate_button.grid(row=0, column=2, padx=10)
translate_button.bind("<Enter>", lambda e: on_enter(translate_button, colors["primary_hover"]))
translate_button.bind("<Leave>", lambda e: on_leave(translate_button, colors["primary"]))

copy_button = tk.Button(action_frame, text="Copy", command=copy_text, font=font_medium, bg=colors["accent"], fg="white", relief="flat", bd=0, width=12, state="disabled")
copy_button.grid(row=0, column=3, padx=10)
copy_button.bind("<Enter>", lambda e: on_enter(copy_button, colors["accent_hover"]))
copy_button.bind("<Leave>", lambda e: on_leave(copy_button, colors["accent"]))

speak_button = tk.Button(action_frame, text="Speak", command=toggle_speech, font=font_medium, bg=colors["speak"], fg="white", relief="flat", bd=0, width=12, state="disabled")
speak_button.grid(row=0, column=4, padx=10)
speak_button.bind("<Enter>", lambda e: on_enter(speak_button, colors["speak_hover"]))
speak_button.bind("<Leave>", lambda e: on_leave(speak_button, colors["speak"]))

clear_button = tk.Button(action_frame, text="Clear", command=clear_text, font=font_medium, bg=colors["secondary"], fg="white", relief="flat", bd=0, width=12)
clear_button.grid(row=0, column=5, padx=10)
clear_button.bind("<Enter>", lambda e: on_enter(clear_button, colors["secondary_hover"]))
clear_button.bind("<Leave>", lambda e: on_leave(clear_button, colors["secondary"]))

history_button = tk.Button(action_frame, text="History", command=show_history, font=font_medium, bg=colors["danger"], fg="white", relief="flat", bd=0, width=12, state="disabled")
history_button.grid(row=0, column=6, padx=10)
history_button.bind("<Enter>", lambda e: on_enter(history_button, colors["danger_hover"]))
history_button.bind("<Leave>", lambda e: on_leave(history_button, colors["danger"]))

# Output section
output_frame = tk.Frame(content, bg=colors["frame_bg"])
output_frame.grid(row=4, column=0, columnspan=2, sticky="ew")
output_label = tk.Label(output_frame, text="Translated Text", font=font_medium, bg=colors["frame_bg"], fg="#495057")
output_label.pack(anchor="w")
output_text_area = tk.Text(output_frame, wrap=tk.WORD, height=8, width=80, font=font_small, bg=colors["output_bg"], fg="#212529", relief="flat", bd=1, highlightthickness=1, highlightbackground="#ced4da", state="disabled")
output_text_area.pack(fill="x", pady=10)
output_scroll = tk.Scrollbar(output_frame, orient="vertical", command=output_text_area.yview)
output_text_area.config(yscrollcommand=output_scroll.set)
output_scroll.pack(side="right", fill="y")
confidence_label = tk.Label(output_frame, text="Confidence: N/A", font=font_small, bg=colors["frame_bg"], fg="#495057")
confidence_label.pack(anchor="w", pady=5)

# Start the main loop
root.mainloop()